In [ ]:
# %matplotlib inline
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os
from scipy.ndimage import gaussian_filter1d as smooth
from scipy.io import loadmat

from aux import loadmat_h5, get_seg, get_outliers, mv_avg
from disp import set_plot, plot_b

T_MAX = 600
T = np.arange(T_MAX)
NROI = 20

T_SINE = np.array([40, 70, 100])
T_PULSE = np.array([130, 160, 190])
T_NAT = np.array([220, 300, 380, 460])

SMOOTH = 3

EXPT = '220815_a1_1'
# EXPT = '220815_a1_2'
# EXPT = '220825_a1_1'
# EXPT = '220825_a1_2'
# EXPT = '220825_a1_3'
# EXPT = '220825_a1_4'
# EXPT = '220825_a1_5'
# EXPT = '220825_a1_6'

DATA_DIR = f'data/original/lin_imaging/nat_song_imaging/{EXPT}/supervoxels/'
FNAME_STIM = 'data/original/lin_imaging/nat_song_imaging/10min_blocksplus_stim_1_forplotting.mat'
SAVE_DIR = f'data/view/lin_20220830'

In [ ]:
stims = loadmat(FNAME_STIM)
stim_pulse = stims['pulse_song'].flatten()
stim_sine = stims['sine_song'].flatten()
tstim = stims['stim_time'].flatten()

mtstim_nat_0 = ((T_NAT[0] - 5) <= tstim) & (tstim < T_NAT[0] + 65)

stim = np.zeros(len(tstim), dtype=int)
stim[stim_sine > 0] = 1
stim[stim_pulse > 0] = 2

In [ ]:
# plot stim
fig, ax = plt.subplots(1, 1, figsize=(15, 2), tight_layout=True)
plot_b(ax, tstim, stim, extent=[0, tstim.max(), 0, 1], c=[(.8, .8, .8), 'b', 'r'])
set_plot(ax, x_label='Time', title='Song', font_size=12)

In [ ]:
for SLICE in range(0, 9):
    FNAME = f'{EXPT}_n20_t600_slice{SLICE}.h5'
    z_dff = loadmat_h5(os.path.join(DATA_DIR, FNAME))['z_dff']

    dffs = []

    for dff in z_dff.T:

        dff[get_outliers(dff, 3)] = np.nan
        dff_smooth = mv_avg(T, dff, SMOOTH)
        
        # z-score
        dff_smooth -= np.nanmean(dff_smooth)
        dff_smooth /= np.nanstd(dff_smooth)

        dffs.append(dff_smooth)

    # plot raw traces
    fig, axs = plt.subplots(NROI, 1, figsize=(15, 2.5*NROI), tight_layout=True)
    for croi, (ax, dff) in enumerate(zip(axs, dffs)):

        lb, ub = np.nanmin(dff), np.nanmax(dff)

        plot_b(ax, tstim, stim, extent=[0, tstim.max(), ub + .2*(ub-lb), ub + .35*(ub-lb)], c=['gray', 'b', 'r'])

        ax.plot(T, dff, c='k')

        for t in T_SINE:
            ax.axvline(t, color='b', ls='--')
        for t in T_PULSE:
            ax.axvline(t, color='r', ls='--')
        for t in T_NAT:
            ax.axvline(t, color='m', ls='--')

        set_plot(ax, x_lim=(0, T_MAX), x_label='Time (s)', y_label='dF/F', title=f'{EXPT} SLICE {SLICE} ROI {croi+1}', font_size=12)
        
    fig.savefig(f'{SAVE_DIR}/raw_{EXPT}_SLICE_{SLICE}.png')

    # plot song-onset-triggered neural responses
    gs = gridspec.GridSpec(NROI, 5)
    fig = plt.figure(figsize=(15, 2.5*NROI), tight_layout=True)
    axs = []

    t_pre = -5
    t_post = 15
    t_block = np.arange(t_pre, t_post)

    t_post_nat = 65
    t_nat = np.arange(t_pre, t_post_nat)

    for croi, dff in enumerate(dffs):
        axs_ = []

        # sine
        ax = fig.add_subplot(gs[croi, 0])
        axs_.append(ax)

        rsps_sine = []

        for t_ in T_SINE:
            mt = ((t_ + t_pre) <= T) & (T < (t_ + t_post))
            rsp_sine = dff[mt] - np.nanmean(dff[mt][t_block < 0])
            ax.plot(t_block, rsp_sine, c='gray')
            rsps_sine.append(rsp_sine)

        ax.plot(t_block, np.nanmean(rsps_sine, axis=0), c='k', lw=4)
        ax.axhline(0, ls='--', color='k')

        lb, ub = np.nanmin(rsps_sine), np.nanmax(rsps_sine)
        plot_b(ax, np.arange(0, 10), np.ones(10), c=['gray', 'b', 'r'], extent=[0, 10, ub + .2*(ub-lb), ub + .3*(ub-lb)])
        set_plot(ax, x_label='Time (s)', y_label='dF/F', title=f'{EXPT}\nSLICE {SLICE} ROI {croi+1}', font_size=12)

        # pulse 
        ax = fig.add_subplot(gs[croi, 1])
        axs_.append(ax)

        rsps_pulse = []

        for t_ in T_PULSE:
            mt = ((t_ + t_pre) <= T) & (T < (t_ + t_post))
            rsp_pulse = dff[mt] - np.nanmean(dff[mt][t_block < 0])
            ax.plot(t_block, rsp_pulse, c='gray')
            rsps_pulse.append(rsp_pulse)

        ax.plot(t_block, np.nanmean(rsps_pulse, axis=0), c='k', lw=3)
        ax.axhline(0, ls='--', color='k')

        lb, ub = np.nanmin(rsps_pulse), np.nanmax(rsps_pulse)
        plot_b(ax, np.arange(0, 10), 2*np.ones(10), c=['gray', 'b', 'r'], extent=[0, 10, ub + .2*(ub-lb), ub + .3*(ub-lb)])
        set_plot(ax, x_label='Time (s)', y_label='dF/F', title=f'{EXPT}\nSLICE {SLICE} ROI {croi+1}', font_size=12)

        # natural song 
        ax = fig.add_subplot(gs[croi, 2:])
        axs_.append(ax)

        rsps_nat = []

        for t_ in T_NAT:
            mt = ((t_ + t_pre) <= T) & (T < (t_ + t_post_nat))
            rsp_nat = dff[mt] - np.nanmean(dff[mt][t_nat < 0])
            ax.plot(t_nat, rsp_nat, c='gray')
            rsps_nat.append(rsp_nat)

        ax.plot(t_nat, np.nanmean(rsps_nat, axis=0), c='k', lw=3)
        ax.axhline(0, ls='--', color='k')

        lb, ub = np.nanmin(rsps_nat), np.nanmax(rsps_nat)
        plot_b(ax, tstim[mtstim_nat_0], stim[mtstim_nat_0], c=['gray', 'b', 'r'], extent=[-5, 65, ub + .2*(ub-lb), ub + .3*(ub-lb)])
        set_plot(ax, x_label='Time (s)', y_label='dF/F', title=f'{EXPT} SLICE {SLICE} ROI {croi+1}', font_size=12)
        
    fig.savefig(f'{SAVE_DIR}/{EXPT}_SLICE_{SLICE}.png')